In [24]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import itertools
from tqdm import tqdm_notebook
import datetime
n_FOLDS = 10
SEED = 888
windows = 14
pd.set_option('display.max_columns',None)
train_2017 = pd.read_csv('../../data/medical/train_2017.csv')
train_2018 = pd.read_csv('../../data/medical/train_2018.csv')
test_2019 = pd.read_csv('../../data/medical/test_2019.csv')
test_2020 = pd.read_csv('../../data/medical/test_2020.csv')
count_2017 = pd.read_csv('../../data/medical/count_2017.csv')
count_2018 = pd.read_csv('../../data/medical/count_2018.csv')
count_2019 = pd.read_csv('../../data/medical/count_2019.csv')
count_2020 = pd.read_csv('../../data/medical/count_2020.csv')
to_predict = pd.read_csv('../../data/medical/to_predict.csv')
submit = pd.read_csv('../../data/medical/submit.csv')

/root/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
illness = to_predict['admin_illness_name'].unique().tolist()

In [26]:
def update_count(count_d, d):
    admin_illness_name = []
    date = []
    count = []
    for x in itertools.product(illness,d):
        admin_illness_name.append(x[0])
        date.append(x[1])
        count.append(0)
    update = pd.DataFrame({'admin_illness_name':admin_illness_name, 'date':date, 'count':count})
    new_count = pd.concat([count_d, update]).drop_duplicates(['admin_illness_name', 'date'],keep='first').sort_values(by=['admin_illness_name', 'date']).reset_index(drop=True)
    return new_count
d_2017 = count_2017[count_2017['admin_illness_name'] == '上呼吸道感染']['date'].tolist()
new_count_2017 = update_count(count_2017, d_2017)    

d_2018 = count_2018[count_2018['admin_illness_name'] == '上呼吸道疾病']['date'].tolist()
new_count_2018 = update_count(count_2018, d_2018)

d_2019 = []
d_2019 += [i for i in range(20190201, 20190229)]
d_2019 += [i for i in range(20190501, 20190532)]
d_2019 += [i for i in range(20190801, 20190832)]
d_2019 += [i for i in range(20191101, 20191131)]
new_count_2019 = update_count(count_2019, d_2019)   

d_2020=[]
d_2020 = [i for i in range(20200301, 20200332)]
d_2020 += [i for i in range(20200601, 20200631)]
new_count_2020 = update_count(count_2020, d_2020)   

In [27]:
count_train = pd.concat([new_count_2017, new_count_2018]).reset_index(drop=True)
count_train = count_train.sort_values(by=['admin_illness_name', 'date']).reset_index(drop=True)

count_test = pd.concat([new_count_2019,new_count_2020]).reset_index(drop=True)
count_test = count_test.sort_values(by=['admin_illness_name', 'date']).reset_index(drop=True)
count_train.shape, count_test.shape

((21900, 3), (5430, 3))

In [28]:
def get_date(data):
    data = data.copy()
    data['illness'] = data['admin_illness_name'].apply(lambda x:illness.index(x)).astype('category')
    data["date"] = data["date"].astype(str)
    data['date_c'] = pd.to_datetime(data['date'])
    data['dayofweek'] = (data['date_c'].dt.dayofweek).astype('category')
    data['month'] = (data['date_c'].dt.month)
    data['is_weekend'] = data['dayofweek'].apply(lambda x:1 if x>4 else 0).astype('category')
    data.rename(columns={'count':'count_his_0'},inplace=True)
    return data
def get_add_data(data):
    o_data = data.copy()
    li = [2,5,8,11,3,6]
    new_data = pd.DataFrame()
    for l in li:
        data = o_data[o_data['month'] == l]
        data['month'] = data['month']
        for i in range(1,windows):
            data['count_his_'+str(i)] = data.groupby(['admin_illness_name'])['count_his_0'].shift(i)
        for i in range(0,windows-1):
            data['count_his_{}_{}'.format(str(i), str(i+1))] = data['count_his_{}'.format(i)] - data['count_his_{}'.format(i+1)]
        for i in range(0,windows-2):
            data['count_his_{}_{}'.format(str(i), str(i+2))] = data['count_his_{}'.format(i)] - data['count_his_{}'.format(i+2)]
        for i in range(1,windows):
            data['count_his_0_{}'.format(str(i))] = data['count_his_0'] - data['count_his_{}'.format(i)]

        "历史统计特征"
        df = pd.DataFrame()
        for i in range(0,windows):
            df = pd.concat([df, data['count_his_'+str(i)]],axis=1)
            if i==2 or i==6 or i==windows-1:
                data['count_his_mean_'+str(i+1)] = df.mean(1)
                data['count_his_median_'+str(i+1)] = df.median(1)
                data['count_his_var_'+str(i+1)] = df.var(1)
        max_count = -1
        min_count = 10000
        li = []
        for i in range(0,windows):
            li += ['count_his_'+str(i)]
        data['count_his_max'] = data[li].max(axis=1)
        data['count_his_min'] = data[li].min(axis=1)

        for i in range(0,7):
            tmp = data.groupby(['date'])['count_his_{}'.format(i)].agg([('all_ill_count_{}'.format(i),'sum')]).reset_index()
            data = pd.merge(data, tmp, on='date', how='left')
            data['ill_all_rate'] = data['count_his_{}'.format(i)] / data['all_ill_count_{}'.format(i)]

        for i in range(1,windows+1):
            data['label_'+str(i)] = data.groupby(['admin_illness_name'])['count_his_0'].shift(-1*i)
            data['log_label_'+str(i)] = np.log(data['label_'+str(i)] + 1)
        new_data = pd.concat([new_data,data])
    new_data.reset_index(inplace=True,drop=True)
    return new_data

In [29]:
def get_data(data, mode='train'):
    data = data.copy()
    "历史count特征" 
    for i in range(1,windows):
        data['count_his_'+str(i)] = data.groupby(['admin_illness_name'])['count_his_0'].shift(i)
    for i in range(0,windows-1):
        data['count_his_{}_{}'.format(str(i), str(i+1))] = data['count_his_{}'.format(i)] - data['count_his_{}'.format(i+1)]
    for i in range(0,windows-2):
        data['count_his_{}_{}'.format(str(i), str(i+2))] = data['count_his_{}'.format(i)] - data['count_his_{}'.format(i+2)]
    for i in range(1,windows):
        data['count_his_0_{}'.format(str(i))] = data['count_his_0'] - data['count_his_{}'.format(i)]
    
    
    "历史统计特征"
    df = pd.DataFrame()
    for i in range(0,windows):
        df = pd.concat([df, data['count_his_'+str(i)]],axis=1)
        if i==2 or i==6 or i==windows-1:
            data['count_his_mean_'+str(i+1)] = df.mean(1)
            data['count_his_median_'+str(i+1)] = df.median(1)
            data['count_his_var_'+str(i+1)] = df.var(1)

    max_count = -1
    min_count = 10000
    li = []
    for i in range(0,windows):
        li += ['count_his_'+str(i)]
    data['count_his_max'] = data[li].max(axis=1)
    data['count_his_min'] = data[li].min(axis=1)
    
    
    "分组特征"
    for i in range(0,7):
        tmp = data.groupby(['date'])['count_his_{}'.format(i)].agg([('all_ill_count_{}'.format(i),'sum')]).reset_index()
        data = pd.merge(data, tmp, on='date', how='left')
        data['ill_all_rate'] = data['count_his_{}'.format(i)] / data['all_ill_count_{}'.format(i)]
        
    
    
    if mode=='test':
        d1 = data[data['date'] == '20190228']
        d2 = data[data['date'] == '20190531']
        d3 = data[data['date'] == '20190831']
        d4 = data[data['date'] == '20191130']
        d5 = data[data['date'] == '20200331']
        d6 = data[data['date'] == '20200630']
        data = pd.concat([d1,d2,d3,d4,d5,d6])
        data = data.sort_values(by=['admin_illness_name', 'date']).reset_index(drop=True)
        return data
    else:
        for i in range(1,windows+1):
            data['label_'+str(i)] = data.groupby(['admin_illness_name'])['count_his_0'].shift(-1*i)
            data['log_label_'+str(i)] = np.log(data['label_'+str(i)] + 1)
        return data

def final_feat(data, day, mode='train'):
    data = data.copy()
    data['month'] = data['month'].astype('category')
    if mode == 'train': 
        col = []
        log_col = []
        new_col = []
        new_log_col = []
        for i in range(1,15):
            col.append('label_' + str(i))
            log_col.append('log_label_' + str(i))  
        use_cols = [i for i in data.columns if i not in col+log_col]
        new_col.append('label_' + str(day+1))
        new_log_col.append('log_label_' + str(day+1))
        data = data[use_cols+new_col+new_log_col]
        data = data.dropna().reset_index(drop=True)
        data['label_{}'.format(day+1)] = data['label_{}'.format(day+1)].astype(int)
        data = data[data['label_{}'.format(day+1)]!=0].reset_index(drop=True)
    for i in range(1,windows):
        data['count_his_'+str(i)] = data['count_his_'+str(i)].astype(int)
    for i in range(1,windows):
        data['count_his_0_{}'.format(str(i))] = data['count_his_0_{}'.format(str(i))].astype(int)
    data['count_his_max'] = data['count_his_max'].astype(int)
    data['count_his_min'] = data['count_his_min'].astype(int)
    return data

train_data = get_date(count_train)   
test_data = get_date(count_test) 

train_test_data = get_add_data(test_data)

train_data = get_data(train_data)
test_data = get_data(test_data, mode='test')
train_data = pd.concat([train_data, train_test_data]).reset_index(drop=True)

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [30]:
def lgb_train(train, test, label, log_label):
    print('训练集大小：',train.shape)
    params_1 = {
        'learning_rate': 0.05,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mae',
        'num_leaves': 31,
        'feature_fraction': 0.7,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'seed': SEED,
        'bagging_seed': 1,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 20,
        'nthread': 3,
        'verbose': -1
    }
    params_2 = {
        'learning_rate': 0.005,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mae',
        'num_leaves': 31,
        'feature_fraction': 0.7,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'seed': SEED,
        'bagging_seed': 1,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 20,
        'nthread': 3,
        'verbose': -1
    }
    feature_importance_df = pd.DataFrame()
    train_preds = np.zeros(train.shape[0])
    test_preds = np.zeros(test.shape[0])
    kfold = KFold(n_splits=n_FOLDS,shuffle=True, random_state=SEED)
    for fold_id, (train_idx, valid_idx) in enumerate(kfold.split(train,log_label)):
        X_train, Y_train = train.iloc[train_idx], log_label.iloc[train_idx]
        X_valid, Y_valid = train.iloc[valid_idx], log_label.iloc[valid_idx]
        dtrain = lgb.Dataset(X_train, Y_train, free_raw_data=False)
        dvalid = lgb.Dataset(X_valid, Y_valid, free_raw_data=False)
        clf = lgb.train(
            params=params_1,
            train_set=dtrain,
            num_boost_round=5000,
            valid_sets=[dtrain,dvalid],
            early_stopping_rounds=50,
            verbose_eval=0)
        
        clf = lgb.train(
            init_model=clf,
            params=params_2,
            train_set=dtrain,
            num_boost_round=5000,
            valid_sets=[dtrain,dvalid],
            early_stopping_rounds=100,
            verbose_eval=0)
        
        train_preds[valid_idx] = clf.predict(X_valid, num_iteration=clf.best_iteration)
        test_preds += clf.predict(test, num_iteration=clf.best_iteration)/n_FOLDS
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = list(X_train.columns)
        fold_importance_df["importance"] = clf.feature_importance(importance_type='gain', iteration=clf.best_iteration)
        fold_importance_df["fold"] = fold_id + 1
    train_preds = np.exp(train_preds) - 1
    test_preds = np.exp(test_preds) - 1
    score = mean_squared_error(train_preds, label) 
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    feature_importance = feature_importance_df.groupby(['feature'])[['importance']].agg('mean').reset_index()
    feature_importance.sort_values(by='importance', ascending=True, inplace=True)
    print(feature_importance)
    print('valid score: ',score)
    print('*'*50)
    return test_preds, score

In [8]:
test_res = pd.DataFrame()
col_n = 'label_'
log_col_n = 'log_label_'
col = []
log_col = []
scores = []
for i in range(1,15):
    col.append(col_n + str(i))
    log_col.append(log_col_n + str(i))  
use_cols = [i for i in train_data.columns if i not in log_col+col+['admin_illness_name','date','date_c']]
for i in tqdm_notebook(range(len(col))):
    n_train_data = final_feat(train_data, i)
    n_test_data = final_feat(test_data, i, 'test')
    test_preds, score = lgb_train(n_train_data[use_cols], n_test_data[use_cols], n_train_data[col[i]], n_train_data['log_'+col[i]])
    test_data[col[i]] = test_preds
    scores.append(score)
test_res = pd.concat([test_res,test_data],axis=0)
test_res.reset_index(inplace=True,drop=True)
print('-'*50)
print(np.mean(scores))

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


训练集大小： (21474, 73)


/root/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/root/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


               feature    importance
71          is_weekend     25.186511
13       count_his_0_2    169.357503
24     count_his_10_12    169.548963
34       count_his_2_3    172.423931
8        count_his_0_1    180.071497
..                 ...           ...
7          count_his_0   7800.170291
58   count_his_mean_14  12189.992721
63  count_his_median_7  19678.212805
59    count_his_mean_3  36574.480515
60    count_his_mean_7  99452.352131

[73 rows x 2 columns]
valid score:  132.35873328740237
**************************************************
训练集大小： (21282, 73)


/root/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/root/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


...

               feature    importance
71          is_weekend    104.982249
14       count_his_0_3    353.149364
45         count_his_6    373.195937
8        count_his_0_1    385.190775
17       count_his_0_6    389.484545
..                 ...           ...
63  count_his_median_7   7901.699117
58   count_his_mean_14  10041.547325
62  count_his_median_3  15813.592746
60    count_his_mean_7  39729.824333
59    count_his_mean_3  77007.114088

[73 rows x 2 columns]
valid score:  299.68736324542283
**************************************************
训练集大小： (19161, 73)


/root/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/root/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


               feature    importance
71          is_weekend    102.044546
14       count_his_0_3    509.298470
45         count_his_6    519.682168
15       count_his_0_4    534.995157
17       count_his_0_6    549.758329
..                 ...           ...
57       count_his_max   7673.861334
63  count_his_median_7   8100.956934
62  count_his_median_3   8768.095850
60    count_his_mean_7  45957.121027
59    count_his_mean_3  74770.982910

[73 rows x 2 columns]
valid score:  314.49920691366947
**************************************************
训练集大小： (18970, 73)


/root/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/root/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


               feature    importance
71          is_weekend     88.181683
14       count_his_0_3    170.508072
56      count_his_9_11    192.707767
52      count_his_8_10    199.842739
53       count_his_8_9    201.771658
..                 ...           ...
62  count_his_median_3  10253.927463
63  count_his_median_7  11405.645621
7          count_his_0  17966.260101
60    count_his_mean_7  21223.684826
59    count_his_mean_3  85900.416056

[73 rows x 2 columns]
valid score:  302.91982491473004
**************************************************

--------------------------------------------------
242.5152318955329


In [18]:
test_res = pd.read_csv('best_result.csv')
t = test_res.copy()

t['count'] = t['count'].astype('int')
t[['id', 'count']].to_csv('result_v.csv',index=False)

In [5]:
to_predict["count"]=0
col_n = 'label_'
col = []
for i in range(1,15):
    col.append(col_n + str(i))
for c in illness:
    f_test=test_data[col].values.flatten()
    to_predict["count"] = f_test
to_predict['count'] = to_predict['count'].apply(lambda x:1 if x<0 else x)
to_predict['count'] = to_predict['count'].astype(int)
to_predict[["id","count"]].to_csv("result_v3.csv",index=False)
to_predict.to_csv('cv_result.csv',index=False)

KeyError: "None of [Index(['label_1', 'label_2', 'label_3', 'label_4', 'label_5', 'label_6',\n       'label_7', 'label_8', 'label_9', 'label_10', 'label_11', 'label_12',\n       'label_13', 'label_14'],\n      dtype='object')] are in the [columns]"